## Sample

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import PIL
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
# Custom transformation for Gaussian noise
class Gaussian(object):
    def __init__(self, mean: float, var: float):
        self.mean = mean
        self.var = var

    def __call__(self, img: torch.Tensor) -> torch.Tensor:
        return img + torch.normal(self.mean, self.var, img.size())

preprocess_augmentation = T.Compose([
    T.CenterCrop((224, 224)),
    # T.Resize((224, 224)),
    T.ToTensor(),
    T.RandomHorizontalFlip(),
    T.RandomRotation(45),
    Gaussian(0, 0.15),
])

preprocess_no_augmentation = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
])


# Dataset class
class MyDataset(Dataset):
    def __init__(self, transform=None, str="train"):
        self.imgs_path = os.path.join("..", "Lab_06", "cats_and_dogs_filtered", str)
        file_list = glob.glob(os.path.join(self.imgs_path, "*"))
        self.data = []
        for class_path in file_list:
            class_name = os.path.basename(class_path)
            for img_path in glob.glob(os.path.join(class_path, "*.jpg")):
                self.data.append([img_path, class_name])
        self.class_map = {"dogs": 0, "cats": 1}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = PIL.Image.open(img_path)
        class_id = self.class_map[class_name]
        class_id = torch.tensor(class_id)
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Train the model
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    train_loader = dataloaders['train']
    val_loader = dataloaders['val']
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Training Phase
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels)
            total_preds += labels.size(0)
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation Phase
        model.eval()
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct_preds += torch.sum(preds == labels)
                total_preds += labels.size(0)

        val_acc = correct_preds.double() / total_preds
        print(f"Val Accuracy: {val_acc:.4f}")

        # Deep copy the model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

# CNN model definition
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(28*28*64, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create datasets with and without augmentation
train_dataset_aug = MyDataset(transform=preprocess_augmentation, str="train")
train_dataset_no_aug = MyDataset(transform=preprocess_no_augmentation, str="train")

# Load datasets into DataLoader
train_loader_aug = DataLoader(train_dataset_aug, batch_size=64, shuffle=True)
train_loader_no_aug = DataLoader(train_dataset_no_aug, batch_size=64, shuffle=True)

# Create validation dataset and dataloaders
val_dataset = MyDataset(transform=preprocess_no_augmentation, str="validation")
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

dataloaders_with_aug = {'train': train_loader_aug, 'val': val_loader}
dataloaders_without_aug = {'train': train_loader_no_aug, 'val': val_loader}


In [ ]:
# Model, Loss function, and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model with data augmentation
print("Training with Data Augmentation")
model_with_aug = train_model(model, dataloaders_with_aug, criterion, optimizer, num_epochs=10)

Training with Data Augmentation
Epoch 0/9
----------
Train Loss: 0.8497 Acc: 0.4960
Val Accuracy: 0.5020
Epoch 1/9
----------
Train Loss: 0.6935 Acc: 0.5000
Val Accuracy: 0.5000
Epoch 2/9
----------
Train Loss: 0.6933 Acc: 0.5000
Val Accuracy: 0.5000
Epoch 3/9
----------
Train Loss: 0.6935 Acc: 0.5045
Val Accuracy: 0.5010
Epoch 4/9
----------
Train Loss: 0.6929 Acc: 0.5185
Val Accuracy: 0.5540
Epoch 5/9
----------
Train Loss: 0.6918 Acc: 0.5195
Val Accuracy: 0.5000
Epoch 6/9
----------
Train Loss: 0.6978 Acc: 0.5175
Val Accuracy: 0.5000
Epoch 7/9
----------
Train Loss: 0.6926 Acc: 0.5135
Val Accuracy: 0.5140
Epoch 8/9
----------
Train Loss: 0.6897 Acc: 0.5445
Val Accuracy: 0.5420
Epoch 9/9
----------
Train Loss: 0.6929 Acc: 0.5120
Val Accuracy: 0.5000
Best val Acc: 0.5540


In [14]:
# Model, Loss function, and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model without data augmentation
print("\nTraining without Data Augmentation")
model_without_aug = train_model(model, dataloaders_without_aug, criterion, optimizer, num_epochs=10)


Training without Data Augmentation
Epoch 0/9
----------
Train Loss: 0.8585 Acc: 0.4980
Val Accuracy: 0.5000
Epoch 1/9
----------
Train Loss: 0.6913 Acc: 0.5235
Val Accuracy: 0.5070
Epoch 2/9
----------
Train Loss: 0.6796 Acc: 0.5680
Val Accuracy: 0.6620
Epoch 3/9
----------
Train Loss: 0.6461 Acc: 0.6300
Val Accuracy: 0.6650
Epoch 4/9
----------
Train Loss: 0.5854 Acc: 0.7000
Val Accuracy: 0.6750
Epoch 5/9
----------
Train Loss: 0.5809 Acc: 0.7050
Val Accuracy: 0.7070
Epoch 6/9
----------
Train Loss: 0.4947 Acc: 0.7605
Val Accuracy: 0.6760
Epoch 7/9
----------
Train Loss: 0.4744 Acc: 0.7720
Val Accuracy: 0.7030
Epoch 8/9
----------
Train Loss: 0.3626 Acc: 0.8475
Val Accuracy: 0.7080
Epoch 9/9
----------
Train Loss: 0.2987 Acc: 0.8780
Val Accuracy: 0.7220
Best val Acc: 0.7220


## Q1
L2 Loss

In [ ]:
# L2 Regularization using loop to find L2 norm of weights
def l2_regularization(model, lambda_l2=0.01):
    l2_norm = 0.0
    for param in model.parameters():
        l2_norm += torch.norm(param, 2) ** 2  # L2 norm squared
    return lambda_l2 * l2_norm

# Modified train_model function to include L2 regularization manually
def train_model_with_l2_regularization(model, dataloaders, criterion, optimizer, num_epochs=10, lambda_l2=0.01):
    train_loader = dataloaders['train']
    val_loader = dataloaders['val']
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Training Phase
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Add L2 regularization
            l2_loss = l2_regularization(model, lambda_l2)
            total_loss = loss + l2_loss  # Add the L2 regularization loss

            total_loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels)
            total_preds += labels.size(0)
            running_loss += total_loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation Phase
        model.eval()
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

            best_acc = val_acc
            best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


In [21]:
# Model, Loss function, and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model with L2 regularization using optimizer's weight decay
print("Training with L2 Regularization (Weight Decay in Optimizer)")
optimizer_with_l2 = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)  # Using weight decay
model_with_l2_optimizer = train_model(model, dataloaders_without_aug, criterion, optimizer_with_l2, num_epochs=10)

Training with L2 Regularization (Weight Decay in Optimizer)
Epoch 0/9
----------
Train Loss: 0.7395 Acc: 0.4875
Val Accuracy: 0.5310
Epoch 1/9
----------
Train Loss: 0.6885 Acc: 0.5410
Val Accuracy: 0.5740
Epoch 2/9
----------
Train Loss: 0.6831 Acc: 0.5650
Val Accuracy: 0.5390
Epoch 3/9
----------
Train Loss: 0.6754 Acc: 0.5735
Val Accuracy: 0.6070
Epoch 4/9
----------
Train Loss: 0.6572 Acc: 0.6055
Val Accuracy: 0.5750
Epoch 5/9
----------
Train Loss: 0.6418 Acc: 0.6385
Val Accuracy: 0.6080
Epoch 6/9
----------
Train Loss: 0.6085 Acc: 0.6665
Val Accuracy: 0.5950
Epoch 7/9
----------
Train Loss: 0.5594 Acc: 0.7210
Val Accuracy: 0.6850
Epoch 8/9
----------
Train Loss: 0.4990 Acc: 0.7580
Val Accuracy: 0.6490
Epoch 9/9
----------
Train Loss: 0.4480 Acc: 0.7945
Val Accuracy: 0.6590
Best val Acc: 0.6850


In [22]:
# Model, Loss function, and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Train model with L2 regularization using manual loop
print("\nTraining with L2 Regularization (Manual Loop for L2 Norm)")
model_with_l2_manual = train_model_with_l2_regularization(model, dataloaders_without_aug, criterion, optimizer, num_epochs=10, lambda_l2=0.0005)


Training with L2 Regularization (Manual Loop for L2 Norm)
Epoch 0/9
----------
Train Loss: 0.8305 Acc: 0.5105
Val Accuracy: 0.5460
Epoch 1/9
----------
Train Loss: 0.7296 Acc: 0.5675
Val Accuracy: 0.5760
Epoch 2/9
----------
Train Loss: 0.7066 Acc: 0.5845
Val Accuracy: 0.5900
Epoch 3/9
----------
Train Loss: 0.6856 Acc: 0.6085
Val Accuracy: 0.5830
Epoch 4/9
----------
Train Loss: 0.6794 Acc: 0.6205
Val Accuracy: 0.5990
Epoch 5/9
----------
Train Loss: 0.6646 Acc: 0.6420
Val Accuracy: 0.6130
Epoch 6/9
----------
Train Loss: 0.6544 Acc: 0.6575
Val Accuracy: 0.6000
Epoch 7/9
----------
Train Loss: 0.6392 Acc: 0.6765
Val Accuracy: 0.6090
Epoch 8/9
----------
Train Loss: 0.5992 Acc: 0.7185
Val Accuracy: 0.6090
Epoch 9/9
----------
Train Loss: 0.5805 Acc: 0.7350
Val Accuracy: 0.6480
Best val Acc: 0.6480


## Q2
L1 Loss

In [29]:
def l1_regularization(model, lambda_l1=0.01):
    l1_norm = 0.0
    for param in model.parameters():
        if param.dim() > 1:  
            l1_norm += torch.sum(torch.abs(param)) 
    return lambda_l1 * l1_norm


# Modified train_model function to include L2 regularization manually
def train_model_with_l1_regularization(model, dataloaders, criterion, optimizer, num_epochs=10, lambda_l1=0.01):
    train_loader = dataloaders['train']
    val_loader = dataloaders['val']
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Training Phase
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            l1_loss = l1_regularization(model, lambda_l1)
            total_loss = loss + l1_loss 

            total_loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels)
            total_preds += labels.size(0)
            running_loss += total_loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation Phase
        model.eval()
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct_preds += torch.sum(preds == labels)
                total_preds += labels.size(0)

        val_acc = correct_preds.double() / total_preds
        print(f"Val Accuracy: {val_acc:.4f}")

        # Deep copy the model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


In [33]:
# Model, Loss function, and Optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("\nTraining with L1 Regularization (Manual Loop for L1 Norm)")
model_with_l1_manual = train_model_with_l1_regularization(model, dataloaders_without_aug, criterion, optimizer, num_epochs=10, lambda_l1=0.0005)


Training with L1 Regularization (Manual Loop for L1 Norm)
Epoch 0/9
----------
Train Loss: 7.0022 Acc: 0.4935
Val Accuracy: 0.5000
Epoch 1/9
----------
Train Loss: 2.6167 Acc: 0.5000
Val Accuracy: 0.5000
Epoch 2/9
----------
Train Loss: 2.3833 Acc: 0.5000
Val Accuracy: 0.5000
Epoch 3/9
----------
Train Loss: 2.3537 Acc: 0.4830
Val Accuracy: 0.5000
Epoch 4/9
----------
Train Loss: 2.3401 Acc: 0.4920
Val Accuracy: 0.5000
Epoch 5/9
----------
Train Loss: 2.3463 Acc: 0.4980
Val Accuracy: 0.5000
Epoch 6/9
----------
Train Loss: 2.3365 Acc: 0.5000
Val Accuracy: 0.5000
Epoch 7/9
----------
Train Loss: 2.3363 Acc: 0.4790
Val Accuracy: 0.5000
Epoch 8/9
----------
Train Loss: 2.3391 Acc: 0.5020
Val Accuracy: 0.5000
Epoch 9/9
----------
Train Loss: 2.3360 Acc: 0.5000
Val Accuracy: 0.5000
Best val Acc: 0.5000


## Q3
Dropouts

In [5]:
class DropoutSimpleCNN(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(DropoutSimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(28 * 28 * 64, 512)
        self.fc2 = nn.Linear(512, 2)

        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = self.fc2(x)
        return x


model = DropoutSimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Training with Dropout")
model_with_dropout = train_model(model, dataloaders_without_aug, criterion, optimizer, num_epochs=10)

Training with Dropout
Epoch 0/9
----------
Train Loss: 0.7917 Acc: 0.4990
Val Accuracy: 0.5000
Epoch 1/9
----------
Train Loss: 0.6929 Acc: 0.5035
Val Accuracy: 0.5050
Epoch 2/9
----------
Train Loss: 0.6930 Acc: 0.5140
Val Accuracy: 0.5080
Epoch 3/9
----------
Train Loss: 0.6926 Acc: 0.5295
Val Accuracy: 0.5550
Epoch 4/9
----------
Train Loss: 0.6860 Acc: 0.5815
Val Accuracy: 0.5320
Epoch 5/9
----------
Train Loss: 0.6738 Acc: 0.5870
Val Accuracy: 0.5680
Epoch 6/9
----------
Train Loss: 0.6703 Acc: 0.6025
Val Accuracy: 0.6150
Epoch 7/9
----------
Train Loss: 0.6246 Acc: 0.6645
Val Accuracy: 0.6320
Epoch 8/9
----------
Train Loss: 0.6057 Acc: 0.6685
Val Accuracy: 0.5720
Epoch 9/9
----------
Train Loss: 0.6157 Acc: 0.6700
Val Accuracy: 0.6390
Best val Acc: 0.6390


## Q4
Custom Dropout

In [6]:
class CustomDropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super(CustomDropout, self).__init__()
        self.p = p
        self.training = True 

    def forward(self, x):
        if self.training:
            mask = torch.bernoulli(torch.full_like(x, 1 - self.p))  
            x = x * mask
            x = x / (1 - self.p) 
        return x

In [ ]:
class CustomDropoutSimpleCNN(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(CustomDropoutSimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(28 * 28 * 64, 512)
        self.fc2 = nn.Linear(512, 2)

        self.dropout = CustomDropoutSimpleCNN(dropout_prob)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = self.fc2(x)
        return x


model = CustomDropoutSimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Training with Dropout")
model_with_dropout = train_model(model, dataloaders_without_aug, criterion, optimizer, num_epochs=10)

## Q5
Early Stopping

In [9]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_wts = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def restore_best_weights(self, model):
        if self.best_model_wts is not None:
            model.load_state_dict(self.best_model_wts)


In [10]:
# Train the model with early stopping
def train_model_with_early_stopping(model, dataloaders, criterion, optimizer, num_epochs=10, patience=5):
    train_loader = dataloaders['train']
    val_loader = dataloaders['val']
    best_model_wts = model.state_dict()
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=patience)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Training Phase
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels)
            total_preds += labels.size(0)
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation Phase
        model.eval()
        correct_preds = 0
        total_preds = 0
        val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                
                _, preds = torch.max(outputs, 1)
                correct_preds += torch.sum(preds == labels)
                total_preds += labels.size(0)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct_preds.double() / total_preds
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Early stopping check
        early_stopping(val_loss, model)
        
        # Stop training if early stopping criteria are met
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch}")
            break

    # Load best model weights and return the model
    early_stopping.restore_best_weights(model)
    return model


In [11]:
# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, loss function, and optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model with early stopping
print("Training with Early Stopping")
model_with_early_stopping = train_model_with_early_stopping(model, dataloaders_without_aug, criterion, optimizer, num_epochs=20, patience=5)


Training with Early Stopping
Epoch 0/19
----------
Train Loss: 0.7856 Acc: 0.4980
Val Loss: 0.6874 Acc: 0.5840
Epoch 1/19
----------
Train Loss: 0.6810 Acc: 0.5820
Val Loss: 0.6772 Acc: 0.5700
Epoch 2/19
----------
Train Loss: 0.6473 Acc: 0.6385
Val Loss: 0.6698 Acc: 0.6080
Epoch 3/19
----------
Train Loss: 0.6059 Acc: 0.6860
Val Loss: 0.6399 Acc: 0.6400
Epoch 4/19
----------
Train Loss: 0.5436 Acc: 0.7420
Val Loss: 0.9029 Acc: 0.5840
Epoch 5/19
----------
Train Loss: 0.5278 Acc: 0.7405
Val Loss: 0.6200 Acc: 0.7000
Epoch 6/19
----------
Train Loss: 0.4665 Acc: 0.7825
Val Loss: 0.6349 Acc: 0.6780
Epoch 7/19
----------
Train Loss: 0.3952 Acc: 0.8235
Val Loss: 0.7291 Acc: 0.6880
Epoch 8/19
----------
Train Loss: 0.3236 Acc: 0.8560
Val Loss: 0.7082 Acc: 0.6920
Epoch 9/19
----------
Train Loss: 0.2555 Acc: 0.8905
Val Loss: 0.8375 Acc: 0.6910
Epoch 10/19
----------
Train Loss: 0.1771 Acc: 0.9285
Val Loss: 0.8701 Acc: 0.6810
Early stopping at epoch 10


In [12]:
# Train the model without early stopping
print("Training without Early Stopping")
model_without_early_stopping = train_model(model, dataloaders_without_aug, criterion, optimizer, num_epochs=20)


Training without Early Stopping
Epoch 0/19
----------
Train Loss: 0.1247 Acc: 0.9600
Val Accuracy: 0.6650
Epoch 1/19
----------
Train Loss: 0.0583 Acc: 0.9810
Val Accuracy: 0.6780
Epoch 2/19
----------
Train Loss: 0.0518 Acc: 0.9835
Val Accuracy: 0.6730
Epoch 3/19
----------
Train Loss: 0.0351 Acc: 0.9870
Val Accuracy: 0.6520
Epoch 4/19
----------
Train Loss: 0.0353 Acc: 0.9880
Val Accuracy: 0.6650
Epoch 5/19
----------
Train Loss: 0.0154 Acc: 0.9970
Val Accuracy: 0.6600
Epoch 6/19
----------
Train Loss: 0.0065 Acc: 1.0000
Val Accuracy: 0.6680
Epoch 7/19
----------
Train Loss: 0.0027 Acc: 1.0000
Val Accuracy: 0.6540
Epoch 8/19
----------
Train Loss: 0.0014 Acc: 1.0000
Val Accuracy: 0.6590
Epoch 9/19
----------
Train Loss: 0.0010 Acc: 1.0000
Val Accuracy: 0.6620
Epoch 10/19
----------
Train Loss: 0.0008 Acc: 1.0000
Val Accuracy: 0.6680
Epoch 11/19
----------
Train Loss: 0.0006 Acc: 1.0000
Val Accuracy: 0.6690
Epoch 12/19
----------
Train Loss: 0.0005 Acc: 1.0000
Val Accuracy: 0.6630
Epo

In [ ]:
# Evaluate the models on validation set
print("Evaluating model with early stopping:")
eval_model(model_with_early_stopping, val_loader)

print("Evaluating model without early stopping:")
eval_model(model_without_early_stopping, val_loader)
